In [1]:
from sklearn import svm
import json
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
with open('training_80%.json','r') as file:
    Train_X_text=json.load(file)
with open('training_80%_labels.json','r') as file:
    Train_Y=json.load(file)

with open('testing_20%.json','r') as file:
    Test_X_text=json.load(file)
with open('testing_20%_labels.json','r') as file:
    Test_Y=json.load(file)

In [3]:
tfidf_transformer = TfidfTransformer(norm = 'l2')
count_vec = CountVectorizer(analyzer="char",max_features = 10000,stop_words='english',ngram_range = (1,8))

trainx_t = count_vec.fit_transform(Train_X_text)
testx_t=count_vec.transform(Test_X_text)

train_x = tfidf_transformer.fit_transform(trainx_t)
test_x= tfidf_transformer.transform(testx_t)

train_x_char=train_x.toarray()
test_x_char=test_x.toarray()

##### tfidf + character n-grams

In [4]:
Random_Forest = RandomForestClassifier(n_estimators=1000, random_state=0,class_weight='balanced') 
Random_Forest.fit(train_x_char,Train_Y)
pred=Random_Forest.predict(test_x_char)
print("Accuracy Score -> ",accuracy_score(pred,Test_Y)*100)

Accuracy Score ->  94.23558897243107


In [9]:
tfidf_transformer = TfidfTransformer(norm = 'l2')
count_vec = CountVectorizer(analyzer="word",max_features = 10000,stop_words='english',ngram_range = (1,2))

trainx_t = count_vec.fit_transform(Train_X_text)
testx_t=count_vec.transform(Test_X_text)

train_x_word = tfidf_transformer.fit_transform(trainx_t)
test_x_word= tfidf_transformer.transform(testx_t)

##### tfidf + word n-grams

In [10]:
Random_Forest = RandomForestClassifier(n_estimators=1000, random_state=0,class_weight='balanced') 
Random_Forest.fit(train_x_word,Train_Y)
pred=Random_Forest.predict(test_x_word)
print("Accuracy Score -> ",accuracy_score(pred,Test_Y)*100)

Accuracy Score ->  82.20551378446115


In [11]:
with open('trainx_emb_80%.json','r') as file:
    Train_X_emb=json.load(file)
with open('testx_emb_20%.json','r') as file:
    Test_X_emb=json.load(file)

In [12]:
train_x_tfidf_elmo=[Train_X_emb[i]+list(train_x_char[i]) for i in range(len(Train_X_emb))]
test_x_tfidf_elmo=[Test_X_emb[i]+list(test_x_char[i]) for i in range(len(Test_X_emb))]

##### elmo embeddings

In [13]:
Random_Forest = RandomForestClassifier(n_estimators=1000, random_state=0,class_weight='balanced') 
Random_Forest.fit(Train_X_emb,Train_Y)
pred=Random_Forest.predict(Test_X_emb)
print("Accuracy Score -> ",accuracy_score(pred,Test_Y)*100)

Accuracy Score ->  80.95238095238095


##### tfidf + elmo embeddings

In [ ]:
#elmo+tfidf+rf
Random_Forest = RandomForestClassifier(n_estimators=1000, random_state=0,class_weight='balanced') 
Random_Forest.fit(train_x_tfidf_elmo,Train_Y)
pred=Random_Forest.predict(test_x_tfidf_elmo)
print("Accuracy Score -> ",accuracy_score(pred,Test_Y)*100)